# Setup

## Code setup

conda create --name codetr python=3.7
conda activate codetr

pip install roboflow wandb

pip install torch==1.11.0+cu113 torchvision==0.12.0+cu113 torchaudio==0.11.0 --extra-index-url https://download.pytorch.org/whl/cu113

pip install mmcv-full==1.5.0 -f https://download.openmmlab.com/mmcv/dist/cu113/torch1.11.0/index.html

git clone https://github.com/Sense-X/Co-DETR.git

cd Co-DETR

pip install -e .

pip install yapf==0.40.1

## Notebook setup

In [ ]:
!pip install roboflow google-cloud-storage

# Get data

Set tokens and dataset version

In [1]:
ROBOFLOW_API_KEY = "YOUR_ROBOFLOW_API_KEY"
ROBOFLOW_PROJECT = "lengoai_nigeria_lagos_apapa_od"
ROBOFLOW_WORKSPACE = "lengo-geomapping"
PROJECT_VERSION = 34

Get data from Roboflow according to project version (which is the version of the dataset created in Roboflow)

In [ ]:
import os
HOME = os.getcwd()

%cd Co-DETR
!mkdir data
%cd data

In [ ]:
from roboflow import Roboflow

rf = Roboflow(api_key=ROBOFLOW_API_KEY)
project = rf.workspace(ROBOFLOW_WORKSPACE).project(ROBOFLOW_PROJECT)
version = project.version(PROJECT_VERSION)
dataset = version.download("coco-mmdetection")

# Get pretrained model

In [ ]:
%cd ..

Obtain the Co-DETR pretrained model on Objects365 dataset

In [17]:
from google.cloud import storage

# Initialize the client
client = storage.Client()

# Define bucket and file
bucket_name = "lengo-geomapping"
pretrained_filename = "models_database/ViT_pretrained_objects365_coco.pth"

bucket = client.get_bucket(bucket_name)
blob = bucket.blob(pretrained_filename)

# Download and read the file
blob.download_to_filename('pretrained/ViT_pretrained_objects365_coco.pth')

# Commands

Firstly, copy the 'model_config.py' script under 'projects/configs/co_dino_vit/' directory.

Then, replace 'coco.py' script in 'Co-DETR/mmdet/datasets/coco.py' to be able to run classwise mAP.

In [ ]:
# Distributed training (single-GPU here)
# bash tools/dist_train.sh projects/configs/co_dino_vit/model_config.py 1 ./test_run

# Testing (single-GPU here)
# python tools/test.py projects/configs/co_dino_vit/model_config.py test_run/<best_model>.pth --show-dir test_run/test --work-dir test_run/test --out test_run/test/output.pkl --eval 'bbox' --cfg-options evaluation.classwise=True

# Confusion matrix
# python tools/analysis_tools/confusion_matrix.py projects/configs/co_dino_vit/model_config.py test_run/test/output.pkl test_run/test --show

# Upload best model to GCS

In [1]:
from google.cloud import storage


def upload_to_gcs(local_file_path, bucket_name, destination_blob_name):
    """
    Uploads a file to a Google Cloud Storage bucket.

    Args:
        local_file_path (str): Path to the local file to upload.
        bucket_name (str): Name of the GCS bucket.
        destination_blob_name (str): Desired filepath in the bucket.
    """
    # Initialize the storage client
    storage_client = storage.Client()

    # Get the bucket
    bucket = storage_client.bucket(bucket_name)

    # Create a blob (object) in the bucket
    blob = bucket.blob(destination_blob_name)

    # Upload the file
    blob.upload_from_filename(local_file_path)

    print(f"File {local_file_path} uploaded to {destination_blob_name} in bucket {bucket_name}.")

In [ ]:
best_model_path = "Co-DETR/test_run/<best_model>.pth"
gcs_destination_path = "models_database/africa/nigeria/best_model_codetr.pth"
bucket_name = "lengo-geomapping"

upload_to_gcs(best_model_path, bucket_name, gcs_destination_path)
